In [ ]:
%load_ext autoreload
from pathlib import Path
import sys
import os
import fitz
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
sys.path.append(str(PROJECT_ROOT.joinpath("src")))
print(f"Project root directory: {PROJECT_ROOT}")


Helper notebook to generate the Catalog from single pages.

In [ ]:
doc_out = fitz.open()

page_counter = 1 # 1-based for bookmarks / toc

page_toc = []

df_dict = {
        "creator": "Reiser Lab",
        "producer": "Janelia Research Campus",
        "creationDate": fitz.get_pdf_now(),
        "modDate": fitz.get_pdf_now(),
        "title": "Cell Type Catalog, Supplement to 'Connectome-driven neural inventory of a complete visual system'",
        "author": "Aljoscha Nern, Laura E. Burnett, Marisa Dreher, Eyal Gruntman, Judith Hoeller, Nathan C. Klapoetke, Sanna Koskela, Frank Loesche, Kit D. Longden, Edward M. Rogers, Pavithraa Seenivasan, Shin-ya Takemura, Arthur Zhao, Sandro Romani, Jan Funke, Stuart Berg, Gerald M. Rubin, Michael B. Reiser",
        "subject": "Connectome",
        "keywords": "connecome, Drosophila, Drosophila melanogaster, vision",
    }

doc_out.set_metadata(df_dict)


os.system('typst compile CellTypeCatalog_Title.typ') # This only works with typst installed. Otherwise it uses the pre-existing pdf file.
title_doc =  "CellTypeCatalog_Title.pdf"
tmp_title = fitz.open(title_doc)
doc_out.insert_pdf(tmp_title)
page_counter += 1




for idx in range(0, 34):
    for fname in ["Gallery_Group-", "Summary_Group-"]:
        tmp_fn = PROJECT_ROOT / "results" / "fig_summary" / f"{fname}{idx:02d}.pdf"
        tmp_doc = fitz.open(tmp_fn)
        doc_out.insert_pdf(tmp_doc)
        tmp_toc = tmp_doc.get_toc(simple=False)
        for toc_entry in tmp_toc:
            toc_e = toc_entry
            toc_e[2] = page_counter
            page_toc.append(toc_e)
        page_counter += 1

doc_out.set_toc(page_toc)

doc_out.save(PROJECT_ROOT / "results" / "fig_summary" / "Cell Type Catalog.pdf")